다나와에서 특정상품(젖병소독기)를 검색한 결과를 가져오는 크롤러입니다.

### 참조 코드 
 
 + 참조 코드입니다.
 + https://blog.naver.com/PostView.nhn?blogId=jaehong7719&logNo=221964071872&parentCategoryNo=&categoryNo=&viewDate=&isShowPopularPosts=false&from=postView
 + 추가

In [5]:
import re, requests, csv
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [3]:
#크롬드라이버가 있는곳의 경로를 chromedriver까지 써줍니다.
driver_path = 'C:/Users/KIMBAEKDU/Downloads/chromedriver_win32 (1)/chromedriver'

### 가격정보 크롤러 (판매중인 쇼핑몰, 할인정보)

In [18]:
##젖병소독기 검색 메인 화면 url
url = 'http://search.danawa.com/dsearch.php?k1=%EC%A0%96%EB%B3%91%EC%86%8C%EB%8F%85%EA%B8%B0&module=goods&act=dispMain'

driver = webdriver.Chrome(driver_path)#크롬드라이버 실행

driver.get(url)#젖병살균기 검색 url로 이동합니다.
time.sleep(4)#사이트 로딩시간 4초

#첫화면에서는 가격비교탭 클릭, 이후에는 2페이지로 이동하는 xpath
page_xpath =  ['//*[@id="productListArea"]/div[1]/div/ul/li[2]/a',
               '//*[@id="productListArea"]/div[4]/div/div/a[2]']

price_compare = [] #크롤링 데이터가 저장되는 리스트입니다.
ap = price_compare.append
for page in page_xpath:#페이지마다 수행합니다.
    time.sleep(3) 
    driver.find_element_by_xpath(page).click() 
    
    time.sleep(3)

    html_list = driver.page_source #페이지 html을 가져와 BeautifulSoup로 다루기 편하게 파싱합니다.
    soup_list = BeautifulSoup(html_list, 'html.parser')
    
    #해당 페이지에 나타난 여러 제품을 클릭했을때 이동하는 주소부분을 리스트로 만듭니다.
    url_info_list = soup_list.find_all('a', {'class' : 'click_log_product_standard_title_'})

    #이 제품목록 주소로 하나하나 이동하면서 크롤링합니다.
    for url in tqdm(url_info_list):
        url_ = re.search('http:.*?\d" ', str(url), re.I|re.S).group()[:-2] #html에서 주소만 남깁니다.
        driver.get(url_)#해당제품 페이지로 이동합니다.
        time.sleep(3)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        name = soup.find('h3', {'class':'prod_tit'})
        name = re.sub('<h3.*?>','', str(name), re.I|re.S)[:-5] #제품명을 따옵니다.
        
        
        try:
            soup_ = soup.find('div', {'id':'defaultMallList'})# 상단에 노출된 광고받은 상품 제외
            
            soup_ = soup_.find_all('div',{'class':'diff_item'}) #각 가격정보, #일시품절상품은 이부분이 없음

            for content in soup_:

                price = content.find('em') #가격부분을 가져옵니다. 가격의 쉼표가 제거되지 않은 상태입니다.
                price = re.sub('<em class.*?>', '', str(price), 0, re.I|re.S)[:-5]

                try:
                    #판매 쇼핑몰 이름을 가져옵니다.
                    shop = content.find_all('a', {'class':'link priceCompareBuyLink'})[0]
                    shop = re.search('<img alt.*? ', str(shop), re.I|re.S).group()[10:-2]
                    
                    #카드 등 할인혜택을 가져옵니다.
                    card = content.find_all('a', {'class':'link priceCompareBuyLink'})[1]
                    card = re.sub('<a class.*?">', '', str(card), 0, re.I|re.S)[:-4]

                except:
                    #판매 쇼핑몰이 일반제휴몰일 경우 예외처리합니다.
                    #카드할인정보가 공백문자열입니다.
                    shop = '일반제휴몰'
                    card = ''
                #모은 정보를 딕셔너리 형태로 저장합니다.
                ap({'name':name,
                    'price' : price,
                    'shop':shop,
                    'card':card})
        except:
            #일시품절상품은 해당 항목이 없으므로 예외처리합니다.
            pass
            
        #뒤로가기로 목록화면으로 이동합니다.
        driver.back()
        
#드라이버 종료
driver.close()


In [19]:
#제대로 크롤링 했는지 확인
price_compare[:5]

[{'name': '쁘띠엘린 모윰 젖병소독기 2세대',
  'price': '160,790',
  'shop': 'G마켓',
  'card': '[KB국민/현대/신한 스마일페이 100만원 이상 결제 시 최대 20개월 무이자]모윰 [모윰] 젖병소독기 2세대 화이트'},
 {'name': '쁘띠엘린 모윰 젖병소독기 2세대',
  'price': '160,790',
  'shop': '옥션',
  'card': '[KB국민/현대/신한 스마일페이 100만원 이상 결제 시 최대 20개월 무이자]젖병소독기 2세대 화이트/ 모윰'},
 {'name': '쁘띠엘린 모윰 젖병소독기 2세대',
  'price': '160,790',
  'shop': '옥션',
  'card': '[KB국민/현대/신한 스마일페이 100만원 이상 결제 시 최대 20개월 무이자]젖병소독기 2세대 컬러선택/ 모윰'},
 {'name': '쁘띠엘린 모윰 젖병소독기 2세대',
  'price': '160,790',
  'shop': '옥션',
  'card': '[KB국민/현대/신한 스마일페이 100만원 이상 결제 시 최대 20개월 무이자]젖병소독기 2세대 그레이/ 모윰'},
 {'name': '쁘띠엘린 모윰 젖병소독기 2세대',
  'price': '160,790',
  'shop': 'G마켓',
  'card': '[KB국민/현대/신한 스마일페이 100만원 이상 결제 시 최대 20개월 무이자]모윰 [모윰] 젖병소독기 2세대 그레이'}]

In [20]:
price_compare = pd.DataFrame(price_compare)
price_compare.tail()

,name,price,shop,card
1789,장안하이텍 소독기 젖병 홀더,"2,880",인터파크,장안 소독기홀더
1790,비엘퓨리티 퓨리티 플러스 젖병 소독기 + 자외선 램프 2개,"200,000",11번가,[삼성카드 1% 청구할인] [신한/KB국민/현대 100만원 이상 SK pay 결제 ...
1791,카리스 자외선 젖병 살균기 KRS-401T,"685,300",인터파크,젖병살균기 KRS-401T 자외선젖병살균기 젖병소독기
1792,에스티전자 해님 BASIC 3세대 젖병 소독기,"187,000",티몬,[해님베이비] 3세대 BASIC 해님 젖병소독기 아기있는집엔 필수
1793,메델라 퀵클린 마이크로 스팀백 5매,"26,190",옥션,[KB국민/현대/신한 스마일페이 100만원 이상 결제 시 최대 20개월 무이자]메델...


In [21]:
#데이터 크기
price_compare.shape

(1794, 4)

In [22]:
#csv로 저장
price_compare.to_csv('danawa_price_compare.csv')

예외처리한 데이터가 있습니다.
추가수정이 필요합니다.

일반제휴쇼핑몰

http://prod.danawa.com/info/?pcode=8054284&amp;keyword=%EC%A0%96%EB%B3%91%EC%86%8C%EB%8F%85%EA%B8%B0&amp;cate=102313 
 
일시품절

http://prod.danawa.com/info/?pcode=3218605&amp;keyword=%EC%A0%96%EB%B3%91%EC%86%8C%EB%8F%85%EA%B8%B0&amp;cate=16321165

### 리뷰정보 크롤러

In [10]:
url = 'http://search.danawa.com/dsearch.php?k1=%EC%A0%96%EB%B3%91%EC%86%8C%EB%8F%85%EA%B8%B0&module=goods&act=dispMain'

driver = webdriver.Chrome(driver_path)#크롬드라이버 실행

driver.get(url)#젖병살균기 검색 url로 이동합니다.
time.sleep(3)#사이트 로딩시간 4초

#첫화면에서는 가격비교탭 클릭, 이후에는 2페이지로 이동하는 xpath
page_xpath =  ['//*[@id="productListArea"]/div[1]/div/ul/li[2]/a',
               '//*[@id="productListArea"]/div[4]/div/div/a[2]']

reviews = [] #리뷰가 모이는 리스트
ap = reviews.append

num1 = 0 #페이지 카운트

for page in page_xpath:#페이지마다 수행합니다.
    time.sleep(3) 
    driver.find_element_by_xpath(page).click() 
    
    time.sleep(3)

    html_list = driver.page_source #페이지 html을 가져와 BeautifulSoup로 다루기 편하게 파싱합니다.
    soup_list = BeautifulSoup(html_list, 'html.parser')
    
    #해당 페이지에 나타난 여러 제품을 클릭했을때 이동하는 주소부분을 리스트로 만듭니다.
    url_info_list = soup_list.find_all('a', {'class' : 'click_log_product_standard_title_'})
    
    
    num1+=1 #페이지 카운트
    print('--------------------'+str(num1)+'페이지 입니다----------------------\n')
    num2 = 0 #제품 카운트
    
    #이 제품목록 주소로 하나하나 이동하면서 크롤링합니다.
    for url in tqdm(url_info_list):
        
        url_ = re.search('http:.*?\d" ', str(url), re.I|re.S).group()[:-2] #html에서 주소만 남깁니다.
        driver.get(url_)#해당제품 페이지로 이동합니다.
        time.sleep(3)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        
        #제품명을 따옵니다.
        product_name = soup.find('h3', {'class':'prod_tit'})
        product_name = re.sub('<h3.*?>','', str(product_name), re.I|re.S)[:-5] 
        
    
        #제품 카운트
        num2+=1
        print(str(num1)+'페이지 '+str(num2)+'번 제품입니다\n')
    
        #총 리뷰 수
        #리뷰가 있을 경우에만 아래 코드 실행
        if bool(soup.find_all('div',{'class':'point_num'})):
            review_count = soup.find('a',{'id':"danawa-prodBlog-companyReview-button-tab-companyReview"})
            review_count = review_count.find('strong', {'class':'num_c'})
            review_count = re.sub('<.+?>', '', str(review_count), re.I|re.S)
            review_count = int(re.sub('[^0-9]','', review_count, re.I|re.S))


            #총 페이지 수
            if review_count % 10 != 0:
                review_page = (review_count // 10) + 1
            else:
                review_page = (review_count // 10)
                
                
            #리뷰 페이지별로 돌아가면서 실행
            for page in tqdm(range(1, review_page+1)):

                time.sleep(2)

                html_for_rvw = driver.page_source
                soup_for_rvw = BeautifulSoup(html_for_rvw, 'html.parser')


                #해당 페이지에 표시된 리뷰들의 리스트
                review_list = soup_for_rvw.find('ul', {'class':'rvw_list'})
                review_list = review_list.find_all('li', {'class':'danawa-prodBlog-companyReview-clazz-more'})


                #리뷰마다 필요한 항목을 뽑아서 딕셔너리로 저장
                for review in review_list:
                    rvw_star = review.find('span', {'class' :'star_mask'})
                    rvw_star = re.sub('<.+?>', '', str(rvw_star), re.I|re.S)

                    rvw_date = review.find('span', {'class' : 'date'})
                    rvw_date = re.sub('<.+?>', '', str(rvw_date), re.I|re.S)

                    rvw_mall = review.find('span', {'class' : 'mall'})
                    rvw_mall = re.sub('<.+?>', '', str(rvw_mall), re.I|re.S)

                    rvw_id = review.find('span', {'class' : 'name'})
                    rvw_id = re.sub('<.+?>', '', str(rvw_id), re.I|re.S)

                    rvw_title = review.find('p', {'class' : 'tit'})
                    rvw_title = re.sub('<.+?>', '', str(rvw_title), re.I|re.S)

                    rvw_content = review.find('div', {'class' : 'atc'})
                    rvw_content = re.sub('<.+?>', '', str(rvw_content), re.I|re.S)


                    #상품명, 별점, 날짜, 쇼핑몰, 아이디, 제목, 내용
                    review_i = {'name':product_name,
                               'star':rvw_star,
                               'date':rvw_date,
                               'mall':rvw_mall,
                               'id':rvw_id,
                               'title':rvw_title,
                               'content':rvw_content}

                    ap(review_i)


                #페이지가 10의 배수일 경우 맨 오른쪽의 넘기기 버튼 클릭
                if page%10 == 0: 
                    path = '//span[@class="point_arw_r"]'
                    button = driver.find_elements_by_xpath(path)[1]
                    button.click()


                #10의 배수가 아닌 페이지일 경우 다음 페이지로 넘기는 버튼 클릭
                else:
                    if page < review_page:
                       path = '//a[@data-pagenumber="{}"]'.format(page+1)
                       button = driver.find_element_by_xpath(path)
                       button.send_keys(Keys.ENTER)    
                    
                    
                    
        #리뷰가 없는 경우
        else:
            print('이 상품은 리뷰가 없습니다')
            pass
        
        #뒤로가기로 목록화면으로 이동합니다.
        driver.back()
        
#드라이버 종료
driver.close()

--------------------1페이지 입니다----------------------



1페이지 1번 제품입니다




1페이지 2번 제품입니다




1페이지 3번 제품입니다




1페이지 4번 제품입니다



...


2페이지 32번 제품입니다




2페이지 33번 제품입니다




2페이지 34번 제품입니다




2페이지 35번 제품입니다




2페이지 36번 제품입니다

이 상품은 리뷰가 없습니다



In [11]:
reviews = pd.DataFrame(reviews)
reviews.tail()

,name,star,date,mall,id,title,content
14925,메델라 퀵클린 마이크로 스팀백 5매,100점,2017.07.17,11번가,dm****,최고예요,휴가철에 놀러가서 쓰려고 주문했어요 ㅎㅎ간편하도 좋네요
14926,메델라 퀵클린 마이크로 스팀백 5매,80점,2016.05.16,11번가,hi****,좋아요!마음에들어요~,놀러갈때 쓰기 딱 좋아영~~
14927,한일전기 한일 젖병 소독기 HBS-900,100점,2019.02.19,G마켓,qu****,잘쓰고있어요,소독할게 많은데 유용하게 잘 쓰고 있어요젖병소독용이라 그런지 좀더 안전하다는 느낌이...
14928,한일전기 한일 젖병 소독기 HBS-900,100점,2018.06.09,SSG.COM,ja****,저렴하게구매했습니다,저렴하게구매했습니다
14929,한일전기 한일 젖병 소독기 HBS-900,40점,2018.06.09,11번가,ru****,별로예요,상품은 좋은ㄷㅔ 물건을 받을때 까지 속 좀 썩네요말도없이 무기한 상품 준비중에 송장...


In [12]:
#데이터 크기
reviews.shape

(14930, 7)

In [17]:
#csv로 저장
reviews.to_csv('danawa_reviews.csv')